In [2]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_road_signs"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_road_signs


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import datetime
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

from skimage import color, exposure

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


In [0]:
train = pd.read_pickle("data/train.p")
test = pd.read_pickle("data/test.p")
train.keys()

X_train, y_train = train['features'], train['labels']
X_train.shape, y_train.shape

X_test, y_test = test['features'], test['labels']
X_test.shape, y_test.shape

signs = pd.read_csv('data/signnames.csv')

dict_signs = signs.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_cnn_v1(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

def train_model(model, X_train, y_train, params_fit={}):    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    
    model.fit(
        X_train,
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks = [tensorboard_callback]
    )
    
    return model
    

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v4(input_shape, num_classes):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),
        
        Dense(1024, activation='relu'),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

In [14]:
model = get_cnn_v4(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

0.9609977324263038

In [0]:
def func_obj(params):
  model = get_cnn_v4(input_shape, num_classes)
  model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose= 0,
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [20]:
space = {
    'barch_size': hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.0537 - accuracy: 1.0000

  8/138 [>.............................]
 - ETA: 0s - loss: 0.0104 - accuracy: 1.0000

 13/138 [=>............................]
 - ETA: 1s - loss: 0.0523 - accuracy: 0.9880

 20/138 [===>..........................]
 - ETA: 0s - loss: 0.1161 - accuracy: 0.9672

 26/138 [====>.........................]
 - ETA: 0s - loss: 0.3760 - accuracy: 0.9387

 32/138 [=====>........................]
 - ETA: 0s - loss: 0.3056 - accuracy: 0.9502


In [0]:
def get_model(input_shape, num_classes, params):
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),
        
        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_one']),
        
        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_two']),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

In [0]:
def func_obj(params):
  model = get_model(input_shape, num_classes, params)
  model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose= 0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [43]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4989196782856521, 'dropout_cnn_block_three': 0.9297754980017084, 'dropout_cnn_block_two': 0.6210753289319206, 'dropout_dense_block_one': 0.17981114113850083, 'dropout_dense_block_two': 0.6214381353116126}
accuracy=0.09773242473602295
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.25178012955630324, 'dropout_cnn_block_three': 0.9878743987185199, 'dropout_cnn_block_two': 0.30197001650889366, 'dropout_dense_block_one': 0.14768416832767906, 'dropout_dense_block_two': 0.04817807542250996}
accuracy=0.054421767592430115
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.9718294209061363, 'dropout_cnn_block_three': 0.5537855726497376, 'dropout_cnn_block_two': 0.46482946903761513, 'dropout_dense_block_one': 0.6605600970461579, 'dropout_dense_block_two': 0.0964720340941303}
accuracy=0.0605442188680172
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.6245811772703457, 'dropout_cnn_block_three': 0.8999335057478136, 'dropout_cnn_block_two': 0.6530583